In [1]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Logistic_Regression_Test_CodeAlong').getOrCreate()

22/09/14 08:02:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
# According to the column counts we are not missing any data. That means we can skip the step of droping the Nan values as they
# simply not there
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [10]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [11]:
from pyspark.ml.feature import VectorAssembler

In [13]:
# Assiging colums to be converted to Vector colum values
assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'
], outputCol='features')

In [14]:
# By creating the output var we are assining the assembler vector values to out orginal dataset: data.
output = assembler.transform(data)

In [16]:
output.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]))

In [21]:
# We are creating a variable containing predictions vector column from out recently transformed output dataset. And the 
# orginal churn column indicating state of our customer
final_data = output.select('features', 'churn')

In [22]:
final_data.head()

Row(features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]), churn=1)

In [23]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [24]:
from pyspark.ml.classification import LogisticRegression

In [28]:
# Creating the Logistic Regression Model
lr_churn = LogisticRegression(labelCol='churn')

In [29]:
# Fitting the Logistic Regression Model into our splitted train data
fitted_churn_model = lr_churn.fit(train_churn)

In [32]:
# Creating var containing the summary of fitted_churn_model. So Fitted Logistic Regression Model and our train data
traning_sum = fitted_churn_model.summary

In [33]:
# We can observe some differences in mean and stddev
traning_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                621|                621|
|   mean|0.16586151368760063|0.12721417069243157|
| stddev| 0.3722561208365632|0.33348134472971436|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [34]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [37]:
# Constructing an evaluation to verify how much our test data matches with our train model 
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [36]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,8670.98,0.0...|    0|[7.47506266410890...|[0.99943327113090...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.51046040937801...|[0.81912942946992...|       0.0|
|[28.0,11128.95,1....|    0|[4.21917833672590...|[0.98550254134918...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.81226902680076...|[0.99701828070522...|       0.0|
|[30.0,10744.14,1....|    1|[1.90243755885451...|[0.87016716052231...|       0.0|
|[30.0,11575.37,1....|    1|[4.05394678135018...|[0.98294226773198...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.35393512734133...|[0.98730705956532...|       0.0|
|[31.0,11297.57,1....|    1|[1.08811398532711...|[0.74802640640400...|       0.0|
|[32.0,5756.12,0.0...|    0|[3.91321693603271...|[0.98041509508017...|       0.0|
|[32.0,8011.38,0

In [40]:
# Creating a BinaryClassificationEvaluator on churn_eval var
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [41]:
# Assining BinaryClassificationEvaluator evaluation object to our pred_and_labels prediction col.
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [43]:
# Now we are able to see the evaluation how in reality our prediction is sufficient 
auc

0.7699926632428468

# Prediction on New Data

In [45]:
# Final data used to construct the final_lr_model is just the churn column and the features vector column established b4.
# Also the lr_churn is just the Logistic Regression set of values of churn col. 
final_lr_model = lr_churn.fit(final_data)

In [46]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [47]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [58]:
# Assining the previously created vector assembler values to new customers dataset
test_new_customer = assembler.transform(new_customers)

In [49]:
test_new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [59]:
# Assining the previously created final_lr_model to new customers dataset
final_results = final_lr_model.transform(test_new_customer)

In [55]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

